# Eval: Sentence Transformer with statement-result set

This notebook outlines the steps undertook to evaluate various sentence transformer models out of the box with the test set under a IR set-up.

With 512 chunk size.

# Import libraries

In [2]:
#%pip install nltk -q
#%pip install sentence_transformers -q

import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Global variable

In [3]:
EVALS = {}

# Helper functions

In [4]:
def split_text_into_chunks(text, max_words=512, overlap_sentences=3):
    sentences = sent_tokenize(text)  # Tokenize into sentences
    chunks = []
    start_idx = 0  # Start index of the current chunk

    while start_idx < len(sentences):
        current_chunk = []
        current_word_count = 0
        idx = start_idx

        # Build a chunk until max_words is reached
        while idx < len(sentences) and current_word_count + len(sentences[idx].split()) <= max_words:
            current_chunk.append(sentences[idx])
            current_word_count += len(sentences[idx].split())
            idx += 1  # Move to next sentence

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        # Stop if the last chunk reaches the end of the text
        if idx >= len(sentences):
            break

        # Move start index forward but keep overlap
        start_idx = max(idx - overlap_sentences, start_idx + 1)

    return chunks


def create_embeddings(df, model, text_column, embedding_column):
    df[embedding_column] = None

    for idx, text in enumerate(df[text_column]):
        chunks = split_text_into_chunks(text)  # Chunk the text
        chunk_embeddings = np.array(model.encode(chunks))  # Encode all chunks (shape: [num_chunks, embedding_dim])
        df.at[idx, embedding_column] = chunk_embeddings

    return df


def evaluate_retrieval(query_emb, doc_embs, relevant_idx, top_k=5):
    # Flatten the document chunks and keep track of the document they belong to
    all_chunk_embeddings = []
    doc_chunk_mapping = []  # Mapping from chunk index to document index
    for doc_idx, doc_chunks in enumerate(doc_embs):
        for chunk_emb in doc_chunks:
            all_chunk_embeddings.append(chunk_emb)
            doc_chunk_mapping.append(doc_idx)  # Store the document index for each chunk

    # Convert to numpy array for efficient computation
    all_chunk_embeddings = np.array(all_chunk_embeddings)  # Shape: [total_chunks, 768]

    # Compute cosine similarity between query and all document chunks
    similarities = np.dot(all_chunk_embeddings, query_emb.T)  # Shape: [total_chunks,]

    # Get indices of top-k most similar chunks
    top_k_chunk_indices = np.argsort(similarities.flatten())[::-1][:top_k]

    # Find the documents corresponding to the top-k chunks
    retrieved_docs = [doc_chunk_mapping[idx] for idx in top_k_chunk_indices]

    # Count the number of times the relevant document appears in the top-k retrieved documents
    relevant_document_hits = sum(1 for doc_idx in retrieved_docs if doc_idx == relevant_idx)

    # Compute rank (based on first appearance of the relevant document)
    rank = None
    for idx, doc_idx in enumerate(retrieved_docs):
        if doc_idx == relevant_idx:
            rank = idx + 1
            break

    # Compute metrics
    is_hit = relevant_document_hits > 0  # Hit if any of the top-k chunks belong to the relevant document
    mrr = 0 if rank is None else 1 / rank

    return {
        "is_hit": is_hit,
        "mrr": mrr,
        "retrieved": retrieved_docs,
        "expected": relevant_idx
    }


def evaluate_model(df, model, query_column, query_emb_column, opinion_column, opinion_emb_column, top_k=5):
    # Step 1: Create embeddings for queries and opinions
    df = create_embeddings(df, model, query_column, query_emb_column)
    df = create_embeddings(df, model, opinion_column, opinion_emb_column)

    # Step 2: Evaluate retrieval for each query
    results_dict = {}
    for idx, query_row in df.iterrows():
        opinion_id = query_row["opinion_id"]
        query_emb = query_row[query_emb_column]
        doc_embs = df[opinion_emb_column]
        results_dict[opinion_id] = evaluate_retrieval(query_emb, doc_embs, idx, top_k=top_k)

    # Step 3: Convert results to DataFrame
    results_df = pd.DataFrame.from_dict(results_dict, orient='index').reset_index().rename(columns={'index': 'opinion_id'})

    # Step 4: Merge results with the original DataFrame
    final_df = df.merge(results_df, how="left", on="opinion_id")

    # Step 5: Calculate evaluation metrics (Hit Rate and MRR)
    model_name = model.name
    EVALS[model_name] = {}
    EVALS[model_name]["hit_rate"] = final_df["is_hit"].mean()
    EVALS[model_name]["mrr"] = final_df["mrr"].mean()

    return final_df

# Load the data

In [5]:
df = pd.read_csv("outputs/3.test.csv")
df = df[["opinion_id", "opinion_word_count", "opinion", "relevant_query_stmt"]]
df.head()

,opinion_id,opinion_word_count,opinion,relevant_query_stmt
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...


# Evaluate model: multi-qa-mpnet-base-dot-v1

In [6]:
%%time

model_name = "multi-qa-mpnet-base-dot-v1"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result1 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result1.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 1min 12s, sys: 2.67 s, total: 1min 15s
Wall time: 1min 18s


,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[-0.0652786, -0.020724855, -0.28577772, -0.27...","[[0.03615426, 0.23663479, -0.18826829, 0.06918...",True,1.0,"[0, 381, 293, 69, 381]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[0.11799486, 0.0074303434, -0.26730403, -0.14...","[[0.22053322, -0.062435195, 0.08577951, -0.176...",True,1.0,"[1, 1, 1, 226, 112]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[0.08234685, -0.059751734, -0.26873463, -0.17...","[[0.2780603, -0.36399046, -0.15213007, -0.1735...",False,0.0,"[165, 425, 382, 79, 375]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[0.10800563, 0.14844967, -0.27977023, -0.1613...","[[0.17971808, -0.11428497, -0.14335318, -0.207...",True,1.0,"[3, 218, 302, 44, 312]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[0.013876454, 0.062817246, -0.35574988, -0.27...","[[0.33654806, 0.28492162, -0.051344726, 0.3195...",False,0.0,"[295, 178, 20, 297, 206]",4


In [7]:
result1.to_csv("4a.eval/IR-512/3.result1.csv", index=False)
len(result1)

450

In [8]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.7822222222222223,
  'mrr': 0.7021851851851852}}

# Evaluate model: all-mpnet-base-v2

In [9]:
model_name = "all-mpnet-base-v2"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result2 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result2.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[0.02522745, 0.051054645, 0.013375721, -0.070...","[[-0.0080017205, 0.06487662, 0.02247683, 0.022...",True,1.000000,"[0, 205, 303, 409, 293]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[0.04486982, 0.02601503, 0.007457674, -0.0628...","[[0.024556143, 0.114506386, 0.013218337, -0.06...",True,1.000000,"[1, 1, 112, 299, 1]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[0.018440673, 0.007830588, -0.015732847, -0.0...","[[0.045272153, -0.013880589, -0.0011564802, -0...",False,0.000000,"[382, 165, 387, 440, 50]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[0.025463922, 0.020822953, -0.0037021146, 0.0...","[[0.023201818, 0.030269556, 0.0069102417, 0.00...",True,0.333333,"[88, 172, 3, 280, 167]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[0.0368914, 0.06671225, -0.0025692736, -0.050...","[[0.035133556, 0.10087562, 0.023342265, 0.0482...",False,0.000000,"[295, 105, 20, 297, 295]",4


In [10]:
result2.to_csv("4a.eval/IR-512/3.result2.csv", index=False)
len(result2)

450

In [11]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.7822222222222223,
  'mrr': 0.7021851851851852},
 'all-mpnet-base-v2': {'hit_rate': 0.7533333333333333,
  'mrr': 0.660074074074074}}

# Evaluate model: multi-qa-distilbert-cos-v1

In [12]:
model_name = "multi-qa-distilbert-cos-v1"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result3 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result3.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[3.83301e-06, -0.02792625, 0.018397603, 0.005...","[[-0.034727223, -0.04516634, -0.011392552, 0.0...",False,0.0,"[444, 381, 381, 381, 381]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[-0.03759421, -0.03136944, 0.04998298, 0.0065...","[[-0.020712376, -0.043643463, -0.0012547055, 0...",True,1.0,"[1, 1, 299, 112, 112]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[-0.055325348, 0.0015135737, -0.013242188, -0...","[[-0.010372783, -0.050516944, -0.033840246, 0....",False,0.0,"[425, 165, 165, 440, 124]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[-0.031819813, -0.025571764, 0.008101494, 0.0...","[[-0.068658866, -0.013247591, -0.016058907, 0....",True,1.0,"[3, 154, 88, 88, 128]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[-0.03216299, -0.025595294, 0.029030858, 0.01...","[[0.026483446, -0.028000247, -0.04994751, 0.00...",False,0.0,"[295, 297, 20, 297, 58]",4


In [13]:
result3.to_csv("4a.eval/IR-512/3.result3.csv", index=False)
len(result3)

450

In [14]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.7822222222222223,
  'mrr': 0.7021851851851852},
 'all-mpnet-base-v2': {'hit_rate': 0.7533333333333333,
  'mrr': 0.660074074074074},
 'multi-qa-distilbert-cos-v1': {'hit_rate': 0.7266666666666667,
  'mrr': 0.6341851851851852}}

# Evaluate model: all-distilroberta-v1

In [15]:
model_name = "all-distilroberta-v1"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result4 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result4.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[-0.007109287, 0.0064733503, -0.013685433, -0...","[[0.000111384004, -0.02229816, -0.023021718, -...",True,1.0,"[0, 293, 381, 303, 90]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[0.012123201, -0.072377376, -0.078425035, 0.0...","[[0.014606625, -0.00021094125, 0.021554783, -0...",True,0.5,"[112, 1, 1, 299, 112]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[-0.0055158567, 0.01545157, -0.0067485236, 0....","[[0.030920794, -0.03444464, -0.011262654, 0.00...",False,0.0,"[95, 231, 18, 165, 342]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[0.014265402, 0.0040137214, 0.012170523, 0.01...","[[0.03038813, 0.026680142, 0.016184725, 0.0171...",True,1.0,"[3, 144, 144, 365, 88]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[0.008720979, -0.01244511, -0.02236831, 0.027...","[[-0.017508037, 0.01724882, 0.019661115, -0.02...",False,0.0,"[295, 295, 297, 20, 20]",4


In [16]:
result4.to_csv("4a.eval/IR-512/3.result4.csv", index=False)
len(result4)

450

In [17]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.7822222222222223,
  'mrr': 0.7021851851851852},
 'all-mpnet-base-v2': {'hit_rate': 0.7533333333333333,
  'mrr': 0.660074074074074},
 'multi-qa-distilbert-cos-v1': {'hit_rate': 0.7266666666666667,
  'mrr': 0.6341851851851852},
 'all-distilroberta-v1': {'hit_rate': 0.7022222222222222,
  'mrr': 0.5958518518518519}}